<a href="https://colab.research.google.com/github/ezzatmohammed/Books-recommendation-sys/blob/main/CMU_Book_Summary_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import libraries**

In [ ]:
import pandas as pd
import numpy
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model

import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from collections import Counter
from pathlib import Path
import os
import numpy as np
import re
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet
import unicodedata
import html
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Load and read df**

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ezzatmohammed","key":"30d0c379ae4f49c883e465e35880fa44"}'}

In [ ]:
#Set Up Kaggle Environment
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#Download the Dataset
!kaggle datasets download -d ymaricar/cmu-book-summary-dataset

Dataset URL: https://www.kaggle.com/datasets/ymaricar/cmu-book-summary-dataset
License(s): CC-BY-SA-3.0
 99% 16.0M/16.2M [00:00<00:00, 17.1MB/s]
100% 16.2M/16.2M [00:00<00:00, 18.0MB/s]


In [ ]:
#Unzip the Dataset
!unzip cmu-book-summary-dataset.zip

Archive:  cmu-book-summary-dataset.zip
  inflating: booksummaries.txt       


In [ ]:
# Read and convert booksummaries.txt to df

df = pd.read_csv('booksummaries.txt', sep='\t', names=['Wikipedia ID', 'Freebase ID',
                                                       'Book Title', 'Author Name', 'Publication Date', 'Genres', 'Summary'])
df.head()

,Wikipedia ID,Freebase ID,Book Title,Author Name,Publication Date,Genres,Summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16559 entries, 0 to 16558
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Wikipedia ID      16559 non-null  int64 
 1   Freebase ID       16559 non-null  object
 2   Book Title        16559 non-null  object
 3   Author Name       14177 non-null  object
 4   Publication Date  10949 non-null  object
 5   Genres            12841 non-null  object
 6   Summary           16559 non-null  object
dtypes: int64(1), object(6)
memory usage: 905.7+ KB


**Date preprocessing**

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.drop(['Wikipedia ID', 'Freebase ID',"Publication Date"], axis=1, inplace=True)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df["Genres"][0]

'{"/m/016lj8": "Roman \\u00e0 clef", "/m/06nbt": "Satire", "/m/0dwly": "Children\'s literature", "/m/014dfn": "Speculative fiction", "/m/02xlf": "Fiction"}'

In [ ]:
import ast

# Function to parse the genre entry
def parse_genre_entry(genre_info):
    # If the genre_info is an empty string, return an empty list
    if genre_info == '':
        return []

    # Convert the genre_info string to a dictionary
    genre_dict = ast.literal_eval(genre_info)

    # Extract the genre names from the dictionary and return them as a list
    genres = list(genre_dict.values())
    return genres

# Apply the function to the 'Genres' column of the DataFrame
df['Genres'] = df['Genres'].apply(parse_genre_entry)
df['Genres']

,Genres
0,"[Roman à clef, Satire, Children's literature, ..."
1,"[Science Fiction, Novella, Speculative fiction..."
2,"[Existentialism, Fiction, Absurdist fiction, N..."
3,"[War novel, Roman à clef]"
4,"[Children's literature, Fantasy, Speculative f..."
...,...
9288,[Science Fiction]
9289,"[Thriller, Fiction, Suspense]"
9290,"[Thriller, Fiction]"
9291,[Autobiography]


In [ ]:
df['Genres'] = df['Genres'].apply(lambda x: ' '.join(x))
df

,Book Title,Author Name,Genres,Summary
0,Animal Farm,George Orwell,Roman à clef Satire Children's literature Spec...,"Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,Anthony Burgess,Science Fiction Novella Speculative fiction Ut...,"Alex, a teenager living in near-future Englan..."
2,The Plague,Albert Camus,Existentialism Fiction Absurdist fiction Novel,The text of The Plague is divided into five p...
3,All Quiet on the Western Front,Erich Maria Remarque,War novel Roman à clef,"The book tells the story of Paul Bäumer, a Ge..."
4,A Wizard of Earthsea,Ursula K. Le Guin,Children's literature Fantasy Speculative fict...,"Ged is a young boy on Gont, one of the larger..."
...,...,...,...,...
9288,The Third Lynx,Timothy Zahn,Science Fiction,The story starts with former government agent...
9289,Remote Control,Andy McNab,Thriller Fiction Suspense,The series follows the character of Nick Ston...
9290,Transfer of Power,Vince Flynn,Thriller Fiction,The reader first meets Rapp while he is doing...
9291,Decoded,Jay-Z,Autobiography,The book follows very rough chronological ord...


In [ ]:
def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))


def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')


def to_lowercase(text):
    return text.lower()



def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
  return word_tokenize(text)

def normalize_text( text):
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    #words = stem_words(words)# Either stem ovocar lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ''.join(words)

In [ ]:
# Apply text normalization to the Summary column
df["Summary"] = df["Summary"].apply(normalize_text)

In [ ]:
# Tokenize the normalized text
from tensorflow.keras.preprocessing.text import Tokenizer

tok = Tokenizer()
tok.fit_on_texts(df["Summary"])

In [ ]:
tok.word_index

{'find': 1,
 'one': 2,
 'take': 3,
 'go': 4,
 'make': 5,
 'life': 6,
 'time': 7,
 'kill': 8,
 'tell': 9,
 'become': 10,
 'return': 11,
 'also': 12,
 'two': 13,
 'new': 14,
 'meet': 15,
 'begin': 16,
 'get': 17,
 'back': 18,
 'leave': 19,
 'father': 20,
 'book': 21,
 'come': 22,
 'see': 23,
 'year': 24,
 'name': 25,
 'story': 26,
 'give': 27,
 'family': 28,
 'first': 29,
 'friend': 30,
 'man': 31,
 'end': 32,
 'know': 33,
 'however': 34,
 'day': 35,
 'novel': 36,
 'help': 37,
 'world': 38,
 'call': 39,
 'work': 40,
 'mother': 41,
 'try': 42,
 'later': 43,
 'use': 44,
 'home': 45,
 'discover': 46,
 'house': 47,
 'love': 48,
 'child': 49,
 'way': 50,
 'death': 51,
 'mr': 52,
 'force': 53,
 'learn': 54,
 'decide': 55,
 'escape': 56,
 'people': 57,
 'attempt': 58,
 'live': 59,
 'woman': 60,
 'reveal': 61,
 'would': 62,
 'war': 63,
 'lead': 64,
 'old': 65,
 'young': 66,
 'follow': 67,
 'attack': 68,
 'order': 69,
 'set': 70,
 'turn': 71,
 'plan': 72,
 'city': 73,
 'send': 74,
 'arrive': 75,


In [ ]:
# Count word frequencies
from collections import Counter


cnt = Counter()
for text in df["Summary"].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)

[('find', 12648),
 ('one', 11300),
 ('take', 11040),
 ('go', 9472),
 ('make', 7979),
 ('life', 7849),
 ('time', 7837),
 ('kill', 7626),
 ('tell', 7578),
 ('become', 7472)]

In [ ]:
# Remove frequent words

FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])

def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["Summary"] = df["Summary"].apply(lambda text: remove_freqwords(text))
df.head()

,Book Title,Author Name,Genres,Summary
0,Animal Farm,George Orwell,Roman à clef Satire Children's literature Spec...,old major old boar manor farm call animal farm...
1,A Clockwork Orange,Anthony Burgess,Science Fiction Novella Speculative fiction Ut...,alex teenager live nearfuture england lead gan...
2,The Plague,Albert Camus,Existentialism Fiction Absurdist fiction Novel,text plague divide five part town oran thousan...
3,All Quiet on the Western Front,Erich Maria Remarque,War novel Roman à clef,book story paul baumer german soldier whourged...
4,A Wizard of Earthsea,Ursula K. Le Guin,Children's literature Fantasy Speculative fict...,ged young boy gont larger island north archipe...


In [ ]:
# Combine text features

df["combined_text"] = df["Summary"] + " " + df["Author Name"] + " " + df["Genres"]

# **Recommendation sys**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Create TF-IDF vectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=1, stop_words='english')
tfidf_matrix = tf.fit_transform(df['combined_text'])

In [ ]:
# Calculate cosine similarity
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# Create a series to map book titles to their index
indices = pd.Series(df.index, index=df['Book Title']).drop_duplicates()

def get_recommendations(title, cosine_sim=cos_sim):
    # Get the index of the book that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar books
    sim_scores = sim_scores[1:11]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar books
    return df['Book Title'].iloc[book_indices]

In [ ]:
print(get_recommendations('The Hunger Games'))

8228               Catching Fire
8755                  Mockingjay
7227    The Will of an Eccentric
6447                The Absolute
2029      Invitation to the Game
2999                   Killobyte
7680                     Century
8771               Virtual World
1463            Lucky Wander Boy
5247         Pop Goes the Weasel
Name: Book Title, dtype: object
